In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import matplotlib.patches
import seaborn as sns

# scikit-learn bootstrap
from sklearn.utils import resample

In [2]:
#PLEASE RUN THIS CELL 
import requests
from IPython.core.display import HTML
styles = requests.get("https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/cs109.css").text
HTML(styles)

### Pre-processing mouse data
Load the clusters obtained from previous section, so that we can bootsrap on them.

In [3]:
### Load DataFrame of log-transfromed averaged time-series for each cluster
# (1) Healthy Group
df_log_healthy = pd.read_pickle("data/df_log_healthy.pkl")
# (2) IBD Group
df_log_ibd = pd.read_pickle("data/df_log_ibd.pkl")

### Load cluster memberships for every OTU
# (1) Healthy Group
tree_healthy = pd.read_pickle( "data/OTU_dm_kmclusters.p")
NMF_healthy = pd.read_pickle( "data/OTU_NMF_healthy.p")
time_healthy = pd.read_pickle( "data/OTU_time_healthy.p")
# (2) IBD Group
tree_ibd = pd.read_pickle( "data/OTU_dm_kmclusters_IBD.p" )
NMF_ibd = pd.read_pickle( "data/OTU_NMF_ibd.p" )
time_ibd = pd.read_pickle( "data/OTU_time_ibd.p")

## Bootstrapping
### Step A. Subset df by cluster Membership
Recall that we have three methods to generate clusters: 
- Tree based: 3 clusters
- NMF correlation: 9 clusters
- Time correlation: 5 clusters

And we have loaded the cluster membership for every OTU above. In this section, we will subset the OTU into those different clusters.

In [131]:
### Function to subset the dataframe by cluster membership
def subset_df_by_membership(df, tree, NMF, time):
    # get the total number of otu and time points
    (otu_length,time_length) = df.shape

    # add the membership as the last column
    df['tree']=tree
    df['NMF']=NMF
    df['time']=time
    
    # loop through 3 different memberships
    methods = ['tree', 'NMF', 'time']
    method_list = list()
    ###########1##############
    # method_list[0]: 'tree' #
    # method_list[1]: 'NMF'  #
    # method_list[2]: 'time' #
    ##########################
    for method in methods:
        # loop through all clusters
        culsters = list(df[method].unique())
        df_list = list()
        #########################2###########################
        # for example:                                      #
        # df_list[0]: OTU with membership as first clusters #
        # ...                                               #
        #####################################################
        for cluster in culsters:
            df_selected = df[df[method] == cluster].iloc[:,:time_length]
            df_list.append(df_selected) #1#
        method_list.append(df_list) #2#
    
    return method_list

In [132]:
### Split the DataFrame into clusters based on their Membership, pack them up into the list
method_list_healthy = subset_df_by_membership(df_log_healthy, tree_healthy, NMF_healthy, time_healthy)
method_list_ibd = subset_df_by_membership(df_log_ibd, tree_ibd, NMF_ibd, time_ibd)

### Step B. Bootstrap to generate more mice data
Now that we have the clusters, we do bootstrap:
- For each single sample step, within every cluster, we randomly choose 30% of the OTUs, took the average of them to generate one time series representing that cluster.
- We repeated the sampling for 30 times, to generate the 30 mice.

In [142]:
### Function to Bootstrap:
def bootrapping(method_list, mice_count):
    methods = list()
    for method in range(3):
        mice = list()
        for time in range(mice_count):
            clusters = list()
            for cluster in range(len(method_list[method])):
                one_sample = method_list[method][cluster].sample(frac=0.3, replace=True)
                log_mean = one_sample[:].mean(axis=0)
                # inverse natural log transform
                real_mean = np.exp(log_mean)
                clusters.append(real_mean)
            mice.append(np.array(clusters))
        methods.append(mice)
        
    tree = methods[0]
    NMF = methods[1]
    time = methods[2]
    return tree, NMF, time

In [150]:
### Generate 30 mice for both group
mice_count=30
# (1) Healthy Mice
tree_healthy_30_mice, NMF_healthy_30_mice, time_healthy_30_mice = bootrapping(method_list_healthy, mice_count)
# (2) IBD Mice
tree_ibd_30_mice, NMF_ibd_30_mice, time_ibd_30_mice = bootrapping(method_list_ibd, mice_count)

### save the mice as a pickle file
# (1) Healthy Mice
pickle.dump(tree_healthy_30_mice, open( "data/30_mice_tree_healthy.p", "wb" ) )
pickle.dump(NMF_healthy_30_mice, open( "data/30_mice_NMF_healthy.p", "wb" ) )
pickle.dump(time_healthy_30_mice, open( "data/30_mice_time_healthy.p", "wb" ) )
# (2) IBD Mice
pickle.dump(tree_ibd_30_mice, open( "data/30_mice_tree_ibd.p", "wb" ) )
pickle.dump(NMF_ibd_30_mice, open( "data/30_mice_NMF_ibd.p", "wb" ) )
pickle.dump(time_ibd_30_mice, open( "data/30_mice_time_ibd.p", "wb" ) )

### Data Structure Example
These are the simulated absolute values (not the log-transformed, they have already been transformed back).

In [151]:
#######################################################################
# For example: tree_healthy_30_mice                                   #
# tree_healthy_30_mice: the first mice data                           #
# tree_healthy_30_mice[0]: the first cluster in the first mice data   #
# tree_healthy_30_mice[0][0]: the 75 time points of the first cluster #
#######################################################################
print('the nunmber of simulated mice data is: ', len(tree_healthy_30_mice))
print('within each mouse, the number of the tree_based clusters is: ', len(tree_healthy_30_mice[0]))
print('for each cluster, the number of the time points is: ', len(tree_healthy_30_mice[0][0]))

the nunmber of simulated mice data is:  30
within each mouse, the number of the tree_based clusters is:  3
for each cluster, the number of the time points is:  75
